In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

import torch

import pyciemss
from pyciemss.PetriNetODE.interfaces import setup_model, reset_model, intervene, sample, calibrate, optimize, load_petri_model
import mira
from pyciemss.utils import get_tspan

import warnings
warnings.filterwarnings('ignore')

In [2]:
from pyciemss.PetriNetODE.interfaces import setup_model, sample, load_petri_model

import mira
from mira.examples.sir import sir

mira_sir = load_petri_model(sir, add_uncertainty=False)


initialized_mira_sir = setup_model(mira_sir, start_time=0, start_state=dict(susceptible_population=99.0,
                                                                            infected_population=1.0,
                                                                            immune_population=0.0))

prior_sample = sample(initialized_mira_sir, timepoints=[1.0, 2.0, 3.0], num_samples=10)
prior_sample

{'immune_population_sol': tensor([[0.0999, 0.1998, 0.2995],
         [0.0999, 0.1998, 0.2995],
         [0.0999, 0.1998, 0.2995],
         [0.0999, 0.1998, 0.2995],
         [0.0999, 0.1998, 0.2995],
         [0.0999, 0.1998, 0.2995],
         [0.0999, 0.1998, 0.2995],
         [0.0999, 0.1998, 0.2995],
         [0.0999, 0.1998, 0.2995],
         [0.0999, 0.1998, 0.2995]]),
 'infected_population_sol': tensor([[0.9990, 0.9978, 0.9966],
         [0.9990, 0.9978, 0.9966],
         [0.9990, 0.9978, 0.9966],
         [0.9990, 0.9978, 0.9966],
         [0.9990, 0.9978, 0.9966],
         [0.9990, 0.9978, 0.9966],
         [0.9990, 0.9978, 0.9966],
         [0.9990, 0.9978, 0.9966],
         [0.9990, 0.9978, 0.9966],
         [0.9990, 0.9978, 0.9966]]),
 'susceptible_population_sol': tensor([[98.9011, 98.8024, 98.7039],
         [98.9011, 98.8024, 98.7039],
         [98.9011, 98.8024, 98.7039],
         [98.9011, 98.8024, 98.7039],
         [98.9011, 98.8024, 98.7039],
         [98.9011, 98.80